# 4. Word Window Classification and Neural Networks 

* http://web.stanford.edu/class/cs224n/lectures/cs224n-2017-lecture4.pdf

In [28]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter
flatten = lambda l: [item for sublist in l for item in sublist]
import sklearn_crfsuite

You also need <a href="http://sklearn-crfsuite.readthedocs.io/en/latest/index.html">sklearn_crfsuite</a> latest version for print confusion matrix

In [30]:
print(torch.__version__)
print(nltk.__version__)

0.2.0+3a8feb7
3.2.2


In [3]:
USE_CUDA = torch.cuda.is_available()

FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor

In [4]:
def getBatch(batch_size,train_data):
    random.shuffle(train_data)
    sindex=0
    eindex=batch_size
    while eindex < len(train_data):
        batch = train_data[sindex:eindex]
        temp = eindex
        eindex = eindex+batch_size
        sindex = temp
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch

In [5]:
def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] if w in word2index.keys() else word2index["<UNK>"], seq))
    return Variable(LongTensor(idxs))

def prepare_word(word,word2index):
    return Variable(LongTensor([word2index[word]]) if word in word2index.keys() else LongTensor([word2index["<UNK>"]]))

def prepare_tag(tag,tag2index):
    return Variable(LongTensor([tag2index[tag]]))

### Data load & processing 

CoNLL-2002 Shared Task: Language-Independent Named Entity Recognition

In [6]:
corpus = nltk.corpus.conll2002.iob_sents()

In [7]:
pdata=[]
for cor in corpus:
    sent,_,tag = list(zip(*cor))
    pdata.append([sent,tag])

In [8]:
print(len(pdata))
print(pdata[0])

35651
[('Sao', 'Paulo', '(', 'Brasil', ')', ',', '23', 'may', '(', 'EFECOM', ')', '.'), ('B-LOC', 'I-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O')]


In [9]:
samples = pdata
#samples = random.sample(pdata,1000) # sampling for practice

In [10]:
sents,tags = list(zip(*samples))
vocab = list(set(flatten(sents)))
tagset = list(set(flatten(tags)))

In [11]:
word2index={'<UNK>' : 0, '<DUMMY>' : 1}
for vo in vocab:
    if vo not in word2index.keys():
        word2index[vo]=len(word2index)
index2word = {v:k for k,v in word2index.items()}

tag2index = {}
for tag in tagset:
    if tag not in tag2index.keys():
        tag2index[tag]=len(tag2index)
index2tag={v:k for k,v in tag2index.items()}

In [12]:
WINDOW_SIZE=2
windows=[]

In [13]:
for sample in samples:
    dummy=['<DUMMY>']*WINDOW_SIZE
    window = list(nltk.ngrams(dummy+list(sample[0])+dummy,WINDOW_SIZE*2+1))
    windows.extend([[list(window[i]),sample[1][i]] for i in range(len(sample[0]))])

In [14]:
windows[0]

[['<DUMMY>', '<DUMMY>', 'Sao', 'Paulo', '('], 'B-LOC']

In [15]:
len(windows)

678377

In [16]:
random.shuffle(windows)

train_data = windows[:int(len(windows)*0.9)]
test_data = windows[int(len(windows)*0.9):]

### Modeling 

In [18]:
class WindowClassifier(nn.Module): 
    def __init__(self,vocab_size,embedding_size,window_size,hidden_size,output_size):

        super(WindowClassifier, self).__init__()
        
        self.embed = nn.Embedding(vocab_size,embedding_size)
        self.h_layer1 = nn.Linear(embedding_size*(window_size*2+1), hidden_size)
        self.h_layer2 = nn.Linear(hidden_size, hidden_size)
        self.o_layer = nn.Linear(hidden_size,output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax()
        self.dropout = nn.Dropout(0.7)
        
    def forward(self, inputs,is_training=False): 
        embeds = self.embed(inputs) # BxWxD
        concated = embeds.view(-1,embeds.size(1)*embeds.size(2)) # Bx(W*D)
        h0 = self.relu(self.h_layer1(concated))
        if is_training:
            h0 = self.dropout(h0)
        h1 = self.relu(self.h_layer2(h0))
        if is_training:
            h1 = self.dropout(h1)
        out = self.softmax(self.o_layer(h1))
        return out

In [64]:
BATCH_SIZE=128
EMBEDDING_SIZE=50 # x (WINDOW_SIZE*2+1) = 250
HIDDEN_SIZE=300
STEP=3
LEARNING_RATE = 0.001

### Training 

It takes a while if you use just cpu.

In [65]:
model = WindowClassifier(len(word2index),EMBEDDING_SIZE,WINDOW_SIZE,HIDDEN_SIZE,len(tag2index))
if USE_CUDA:
    model = model.cuda()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=LEARNING_RATE)

In [66]:
for step in range(STEP):
    losses=[]
    for i,batch in enumerate(getBatch(BATCH_SIZE,train_data)):
        x,y=list(zip(*batch))
        inputs = torch.cat([prepare_sequence(sent,word2index).view(1,-1) for sent in x])
        targets = torch.cat([prepare_tag(tag,tag2index) for tag in y])
        model.zero_grad()
        preds = model(inputs,is_training=True)
        loss = loss_function(preds,targets)
        losses.append(loss.data.cpu().numpy()[0] if USE_CUDA else loss.data.numpy()[0] )
        loss.backward()
        optimizer.step()

        if i % 1000==0:
            print("[%d/%d] mean_loss : %0.2f" %(step,STEP,np.mean(losses)))
            losses=[]

[0/3] mean_loss : 2.10
[0/3] mean_loss : 0.53
[0/3] mean_loss : 0.45
[0/3] mean_loss : 0.39
[0/3] mean_loss : 0.35
[1/3] mean_loss : 0.21
[1/3] mean_loss : 0.29
[1/3] mean_loss : 0.28
[1/3] mean_loss : 0.26
[1/3] mean_loss : 0.25
[2/3] mean_loss : 0.18
[2/3] mean_loss : 0.22
[2/3] mean_loss : 0.21
[2/3] mean_loss : 0.21
[2/3] mean_loss : 0.20


### Test 

In [67]:
for_f1_score=[]

In [68]:
accuracy=0
for test in test_data:
    x,y = test[0],test[1]
    input_ = prepare_sequence(x,word2index).view(1,-1)

    i = model(input_).max(1)[1]
    pred = index2tag[i.data.cpu().tolist()[0]] if USE_CUDA else index2tag[i.data.tolist()[0]]
    for_f1_score.append([pred,y])
    if pred==y:
        accuracy+=1

print(accuracy/len(test_data)*100)
# print(index2tag[i.data.cpu().tolist()[0]] if USE_CUDA else index2tag[i.data.tolist()[0]] )
# print(y)

94.45148736696247


This high score is because most of labels are 'O' tag. So we need to measure f1 score.

### Print Confusion matrix 

In [69]:
y_pred, y_test = list(zip(*for_f1_score))

In [70]:
sorted_labels = sorted(
    list(set(y_test)-{'O'}),
    key=lambda name: (name[1:], name[0])
)

In [71]:
sorted_labels

['B-LOC', 'I-LOC', 'B-MISC', 'I-MISC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER']

In [72]:
y_pred = [[y] for y in y_pred] # this is because sklearn_crfsuite.metrics function flatten inputs
y_test = [[y] for y in y_test]

In [73]:
print(sklearn_crfsuite.metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

             precision    recall  f1-score   support

      B-LOC      0.728     0.559     0.632      1095
      I-LOC      0.875     0.023     0.045       304
     B-MISC      0.656     0.233     0.344       785
     I-MISC      0.448     0.040     0.074       647
      B-ORG      0.822     0.577     0.678      1422
      I-ORG      0.589     0.650     0.618       996
      B-PER      0.773     0.659     0.711      1278
      I-PER      0.790     0.795     0.793       959

avg / total      0.717     0.521     0.571      7486

